# Importing Libraries

In [15]:
import os
import dlib
import cv2
from imutils import face_utils
import re
import numpy as np
import math

# Read dataset

In [16]:
def atoi(text):
    if text.isdigit():
        return int(text) 
    else:
        return text

def natural_keys(text):
    l=[]
    for c in re.split('(\d+)', text):
       l.append(atoi(c))     
    return(l)    

dataset_path = "faces"
file_list = os.listdir(dataset_path)
file_list = [x for x in file_list if re.search("^.*pgm$", x)]
file_list.sort(key = natural_keys)

def get_subject_name(subject):
    if subject < 10:
        subject = "0" + str(subject)
    else:
        subject = str(subject)
    return subject

def get_subject_images(subject, file_list):
    return [x for x in file_list if re.search("^.*" + subject + ".*pgm$", x)]

# Euclidean Distance

In [17]:
def euclidean_distance(x, y):
    return math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))

# Initializing Variables

In [18]:
indices = [2, 0, 8, 7, 10, 4, 9, 1, 5, 6, 3]

split = 8
no_of_classes = 15
images_per_subject = 11
no_of_landmarks =  68

predictor_path = "shape_predictor_68_face_landmarks.dat"


# Retrieving dlib's detector and predictor

In [10]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Train and Test split

In [11]:
Train = np.zeros((no_of_classes * split, 2))
Test = np.zeros((no_of_classes * (images_per_subject - split), 2))
train = 0
test = 0
for subject in range(1, no_of_classes + 1):
    
    subject = get_subject_name(subject)
    subject_landmarks = []
    images_list = get_subject_images(subject, file_list)
    images_list = [images_list[i] for i in indices]
    for image_name in images_list:
    
        image_path = dataset_path + "\\" + image_name
        image = cv2.imread(image_path,-1)
        faces_in_image = detector(image, 1)
        
        for i, face in enumerate(faces_in_image):
            shape = predictor(image, face)
            shape = face_utils.shape_to_np(shape)
                
        representative_landmark = [ sum([point[0] for point in shape]) / 68, sum([point[1] for point in shape]) / 68]
        
        if images_list.index(image_name) < split:
            Train[train,:] = representative_landmark
            train += 1
        else:
            Test[test,:] = representative_landmark
            test += 1

# Classification

In [12]:
count = 0
img_count = 0
for test_landmark in Test:
    img_count += 1
    distances = []
    for subject_landmark in Train:
        dist = euclidean_distance(subject_landmark, test_landmark)
        distances.append(dist)
    
    if int(img_count//3) == int(distances.index(min(distances)) // split):
        count +=  1

# Result

In [13]:
print("Accuracy:",count*100/img_count, " %") 

Accuracy: 33.333333333333336  %
